## 라이브러리

In [11]:
import time
# Selenium 모듈에서 webdriver를 가져옵니다.
from selenium import webdriver 
# Chrome 브라우저 옵션을 다루는 Options 클래스를 가져옵니다.
from selenium.webdriver.chrome.options import Options
# Chrome 드라이버 서비스를 다루는 Service 클래스를 가져옵니다.
from selenium.webdriver.chrome.service import Service
# Chrome 드라이버를 다운로드하고 설치하는 데 사용되는 ChromeDriverManager를 가져옵니다.
from webdriver_manager.chrome import ChromeDriverManager

 
# Options 클래스의 인스턴스를 생성합니다.
options = Options()
 
# Chrome 브라우저 창이 즉시 닫히는 것을 방지합니다.
options.add_experimental_option('detach', True)
 
# 불필요한 콘솔 메시지를 제거합니다.
options.add_experimental_option('excludeSwitches', ['enable-logging'])
 
# Chrome 드라이버 서비스를 생성합니다.
service = Service(ChromeDriverManager().install())

## 네이버 api

In [12]:
from PyNaver import Naver

from dotenv import load_dotenv
import os
import random

load_dotenv()

# 애플리케이션 인증 정보
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

# 네이버 API 인스턴스 생성
api = Naver(client_id, client_secret)

## Shop api 추출

In [13]:
from bs4 import BeautifulSoup
import requests

def getdata_naverapi(url, keyword, params):
    headers = {'X-Naver-Client-Id': client_id,
               'X-Naver-Client-Secret': client_secret}

    r = requests.get(url, params=params, headers=headers)
    rescode = r.status_code
    
    if rescode == 200:
        j = r.json()
        # for i, item in enumerate(j['items'][:1], start=1):
        #     print(i, item)
        return j
    else:
        print("Error Code:", rescode)



url = 'https://openapi.naver.com/v1/search/shop.json' # api 주소
keyword = '정관장' # 키워드
# s = 1, d = 100, 100 
# s = 101, d = 100, 201
# s = 201, d = 100, 301
# ~~ 
# s = 901, d = 100, 1001
# s = 1000, d = 100, 1100
total_params = []
for s in range(1,1001,100):
    d = 100
    params = {'query': keyword,
              'display': d,
              'start': s,
              'sort': 'sim'} #파라미터
    total_params.append(params) 

d = 100
params = {'query': keyword,
            'display': d,
            'start': 1000,
            'sort': 'sim'} #파라미터
total_params.append(params) 


# title link image Iprice hprice mallName 
# productId productType brand maker category1~4
data = []
for params in total_params:
    result = getdata_naverapi(url, keyword, params)
    if result:
        for i, item in enumerate(result['items'], start=1):
            name = item['title']
            lprice = item['lprice']
            hprice = item['hprice']
            link = item['link']
            mallName = item['mallName']
            maker = item['maker']
            brand = item['brand']
            category1 = item['category1']
            category2 = item['category2']
            category3 = item['category3']
            category4 = item['category4']
            product_naver_id = item['productId']
            
            soup = BeautifulSoup(name, 'html.parser')
            cl_name = soup.get_text()
            dic = {
                'name': cl_name,
                'lprice': lprice,
                'hprice': hprice,
                'link': link,
                'mallName': mallName,
                'maker': maker,
                'brand': brand,
                'category1': category1,
                'category2': category2,
                'category3': category3,
                'category4': category4,
                'productId': product_naver_id
            }
           
            data.append(dic)

/var/folders/wf/_zt37wm525967j6wl83b93mw0000gn/T/ipykernel_44757/144512078.py:66: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(name, 'html.parser')


In [14]:
# import pandas as pd
# pd.DataFrame(data)

## 링크만

In [15]:
links = [item['link'] for item in data]
links

['https://search.shopping.naver.com/gate.nhn?id=11259655619',
 'https://search.shopping.naver.com/gate.nhn?id=5944644989',
 'https://search.shopping.naver.com/gate.nhn?id=82064672715',
 'https://search.shopping.naver.com/gate.nhn?id=5948365403',
 'https://search.shopping.naver.com/gate.nhn?id=9689425728',
 'https://search.shopping.naver.com/gate.nhn?id=6663522597',
 'https://search.shopping.naver.com/gate.nhn?id=9255057462',
 'https://search.shopping.naver.com/gate.nhn?id=12022041834',
 'https://search.shopping.naver.com/gate.nhn?id=14758481330',
 'https://search.shopping.naver.com/gate.nhn?id=5770614300',
 'https://search.shopping.naver.com/gate.nhn?id=17789393164',
 'https://search.shopping.naver.com/gate.nhn?id=22188734073',
 'https://search.shopping.naver.com/gate.nhn?id=5594729682',
 'https://search.shopping.naver.com/gate.nhn?id=14240053390',
 'https://search.shopping.naver.com/gate.nhn?id=5769800164',
 'https://search.shopping.naver.com/gate.nhn?id=6275470973',
 'https://search.

In [31]:
# 테스트용 링크 데이터 샘플 3개
test_link = links[:2].copy()
test_link = ['https://search.shopping.naver.com/product/32460672623']

## 링크 데이터 추출

In [32]:
from selenium.webdriver.common.by import By
import re

# 리뷰 데이터 추출
def score_extract(element):
    return int(re.findall(r'\d+', element.text)[0])

def idx_extract(infos):
    try:
        try:
            if infos.text[-1] == '*':
                idx = True
            else:
                idx = False
        except:
            idx = False
        return idx
    except:
        # print(infos.text)
        return 'error'

def text_extract(infos):
    return infos.text

def extract_review_data(reviews):
    # 평점
    scores = reviews.find_elements(By.CLASS_NAME, "reviewItems_average__0kLWX")
    scores = list(map(score_extract, scores))
    
    # 리뷰에서 스토어, ID, DATE 추출
    infos = reviews.find_elements(By.CLASS_NAME, "reviewItems_etc__9ej69")

    # print(infos)
    id_idxs = list(map(idx_extract, infos))
    infos  = list(map(text_extract, infos))

    ids = []
    dates = []
    
    for id, b in enumerate(id_idxs):
        if b == True:
            ids.append(infos[id])
            dates.append(infos[id+1])
    
    # 리뷰에서 데이터 추출
    datas = reviews.find_elements(By.CLASS_NAME, "reviewItems_text__XrSSf")
    datas  = list(map(text_extract, datas))
    return list(zip(ids, scores, dates, datas))

# 활성 페이지 찾기
def search_index(a_list):
    for idx, a in enumerate(a_list):
        try:
            now = [True, a.find_element(By.TAG_NAME, 'span')]
            return idx
        except:
            now = [False, []]

In [11]:
import pandas as pd
# pd.read_csv('../last/data/Data_Product.csv', sep='\t')
pd.read_csv('../last/data/Data_Reviews.csv', sep='\t')

,product_id,id,score,date,data,search_id
0,10,b*i*****,5,22.10.13.,*마트서 하나 사와서 먹길래 샤인머스캣맛에 혹해서 한번 맛보고는 넘 맛있어서 박스로...,1
1,10,la******,5,23.05.17.,십일절이 5일이라 좋으네요~^^ 덕분에 뽀로로음료수를 두박스 구매할 수 있었어요!!...,1
2,10,so******,5,23.03.25.,애가 입이 짧아 늘 밀크맛만 마셨는데 친구맘이 샤인머스켓을 하나 주셨어요~ 어쩌려나...,1
3,10,pzzo****,5,22.06.08.,아이가 너무 좋아해요.어린이날 간식주머니이 하나씩 넣어보냈어요.뽀로로 쥬스 여러가지...,1
4,10,pzzo****,5,22.04.21.,아이가 너무 좋아해요. 최애는 사과랑 샤인머스켓이에요~ 유통기한도 내년 4월~넉넉하...,1
...,...,...,...,...,...,...
264,10,kyog****,5,22.02.14.,잘받았습니다.잘받았어요,1
265,10,sky*****,5,21.11.06.,만족합니다만족합니다,1
266,10,wjds****,4,22.09.25.,잘받았습니다 잘받았습니다,1
267,10,s*m*****,5,22.01.13.,좋습니다~ 좋습니다~ㅋ,1


In [33]:
# 링크입력
driver = webdriver.Chrome(service=service, options=options)
review_ls_list = []
for l in test_link:
    driver.get(l)
    time.sleep(random.uniform(2,4))

    # 사러가기 클릭
    # xpath = '//*[@id="wrap"]/div[2]/a[2]'
    # element = driver.find_element(By.XPATH, xpath)
    # category_1 = {idx:name for idx, name in enumerate(element.text.split('\n'), start=1)}
    # category_1

    xpath = '//*[@id="wrap"]/div[2]/a[2]'
    element = driver.find_element(By.XPATH, xpath)
    element.click()

    while True:
        # 리뷰 엘리먼트 탐색
        time.sleep(random.uniform(0.4,0.7))
        # start_time = time.time() # 시작
        xpath = '//*[@id="section_review"]/ul'
        reviews = driver.find_element(By.XPATH, xpath)
        
        review_ls_list += extract_review_data(reviews)

        # 페이지 넘기기
        xpath = '//*[@id="section_review"]/div[3]'
        page_views = driver.find_element(By.XPATH, xpath)
        a_list = page_views.find_elements(By.TAG_NAME, 'a')

        idx = search_index(a_list)
        page_num = a_list[idx].text.split('\n')[1]
        # print(f"{page_num}. {time.time()-start_time:.4f} sec") # 종료와 함께 수행시간 출력
        # print(f"{page_num} {time.time()-start_time:.4f} sec") # 종료와 함께 수행시간 출력
        print('Page', page_num)
        if int(page_num) == 100:
            break
        if (len(a_list)-1) != idx:
            # print(a_list[idx].text.split('\n')[1])
            a_list[idx+1].click()
        else:
            print('end')
            break

driver.close()

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x0000000100d6f8ac chromedriver + 4257964
1   chromedriver                        0x0000000100d67f40 chromedriver + 4226880
2   chromedriver                        0x00000001009a49d4 chromedriver + 281044
3   chromedriver                        0x000000010097edb0 chromedriver + 126384
4   chromedriver                        0x0000000100a03938 chromedriver + 670008
5   chromedriver                        0x0000000100a15fe8 chromedriver + 745448
6   chromedriver                        0x00000001009d398c chromedriver + 473484
7   chromedriver                        0x00000001009d498c chromedriver + 477580
8   chromedriver                        0x0000000100d2e900 chromedriver + 3991808
9   chromedriver                        0x0000000100d32354 chromedriver + 4006740
10  chromedriver  

In [20]:
len(review_ls_list)

4000

In [21]:
#중복값 제거
unique_reviews = list(set(map(tuple, review_ls_list)))
len(unique_reviews)

3996

In [29]:
def ss(row):
    return row[0]+row[2]

In [30]:
pd.DataFrame(unique_reviews).apply(ss)

0                                     bang****onoy****
1                                                    9
2                                   23.02.05.22.08.31.
3    엄마가 정년퇴직을 앞두고 기력이 많이 쇠하셔서 처음으로 홍삼을 사드려봤어요. 처음부...
dtype: object

In [22]:
unique_reviews

[('bang****',
  5,
  '23.02.05.',
  '엄마가 정년퇴직을 앞두고 기력이 많이 쇠하셔서 처음으로 홍삼을 사드려봤어요. 처음부터 너무 액기스를 사드리면 부담스러워하실거 같아서 에브리타임으로 사다드렸는데 딱 좋다고 하시네요! 매일매일 드신 후로 감기도 한번도 안 걸리셨고 면역력을 많이 되찾은 느낌이라고 하십니다 :) 추천합니다!'),
 ('ashk****', 5, '23.01.14.', '선물용으로 잘샀어요ㅎㅎ포장 너무 이쁘네요'),
 ('onoy****',
  4,
  '22.08.31.',
  '먹기 편해서 좋고 제품퀄리티도 좋은편입니다.\n가격은 좀 비싼편이긴 하나 ,, 선물로도 좋은듯합니다.'),
 ('g*e*****', 5, '21.06.28.', '부모님 선물로 드리면 정말 좋아하시네요.'),
 ('sege****',
  5,
  '22.05.07.',
  '아침저녁으로 진한맛 느끼고 있어요.\n쓰지만 농축되었으리라 믿고 열심히 챙겨 먹어볼게요!'),
 ('alic****', 5, '21.10.19.', '빠른배송 감사합니다. 겨울앞두고 한번씩 먹어줘야함.'),
 ('a*g*****', 5, '21.03.07.', '추천합니다 배송도 빠르고 만족합니다'),
 ('pjjy****', 5, '22.08.10.', '면역력에 좋다하니 구매했습니다\n빠른 배송 감사합니다 수고하세요'),
 ('ssan****', 5, '21.08.15.', '선물드린건데\n아주 만족하신다고 하시네요\n좋은 상품 저렴하게 잘 샀어요'),
 ('bawo****', 5, '22.12.02.', '재구매했구요. 다 먹고 또 구매할게요.'),
 ('dewm****', 5, '21.09.22.', '정관장 항상 애용하고 믿고사는 제품입니다\n선물로도 최고!'),
 ('kimw****',
  5,
  '21.03.23.',
  '친정엄마가 잘 못드시고 힘이 없다하셔서 주문해드렸어요\n간편하게 매일 챙겨드시기 좋다고 하시네요\n면역력도 높여주고 활기있게 해드렸음 좋겠

In [63]:
import pandas as pd

df = pd.DataFrame(unique_reviews)
df.to_excel('리스트.xlsx')